In [37]:
import tensorflow as tf
import numpy as np
import os
from lxml import etree
import glob
import cv2

剪裁图片

In [ ]:
# 定义剪裁人脸函数 

def tailor_img(image, xml):
    xml = open('{}'.format(xml),encoding='utf-8').read()
    sel = etree.HTML(xml)
    xmin = int(sel.xpath('//bndbox/xmin/text()')[0])
    xmax = int(sel.xpath('//bndbox/xmax/text()')[0])
    ymin = int(sel.xpath('//bndbox/ymin/text()')[0])
    ymax = int(sel.xpath('//bndbox/ymax/text()')[0])
    
    img = cv2.imread(image)
    dst = img[ymin:ymax, xmin:xmax]
    
    return dst

In [10]:
# 获取图片和xmls路径

images = glob.glob('D:/Jupyter/dataset/face2/data/*.jpg')
xmls = glob.glob('D:/Jupyter/dataset/face2/label/*.xml')

In [12]:
# 图片和xmls数量不对，筛选一下
# 获取文件名
img_name = [img.split('\\')[1].split('.jpg')[0] for img in images]
xml_name = [x.split('\\')[1].split('.xml')[0] for x in xmls]

In [14]:
len(img_name), len(xml_name)

(3539, 3268)

In [15]:
images2 = []
for x in images:
    if int(x.split('\\')[1].split('.jpg')[0] in xml_name)==1:
        images2.append(x)

In [16]:
img_name2 = [img.split('\\')[1].split('.jpg')[0] for img in images2]

In [17]:
xmls2 = []
for x in xmls:
    if int(x.split('\\')[1].split('.xml')[0] in img_name2)==1:
        xmls2.append(x)

In [18]:
# 经过一顿操作现在数量相同了
len(images2), len(xmls2)

(3261, 3261)

In [ ]:
# 剪裁出来人脸

s = len(images2)
img_data = []   # 人脸图片
new_images = images2.copy()   #原图片路径
for i in range(s):
    data = tailor_img(images2[i], xmls2[i])
    img_data.append(data)


In [ ]:
# 保存人脸图片到本地

for i in range(len(img_data)):
    path = 'D:/Jupyter/dataset/face2/face_img/'
    name = os.path.basename(images[i])
    pathN = os.path.join(path, name)
    cv2.imwrite(pathN,img_data[i])

In [19]:
# 获取人脸图片
face_img = glob.glob('D:/Jupyter/dataset/face2/face_img/*.jpg')

In [20]:
# 定义获取图片对应的label函数
def get_label(xml):
    xml = open('{}'.format(xml),encoding='utf-8').read()
    sel = etree.HTML(xml)
    label = sel.xpath('//object/name/text()')[0]
    return label

In [21]:
labels = []
for i in xmls2:
    l = get_label(i)
    if l=='nomask':
        l = 0
    else:
        l = 1
    labels.append(l)

In [22]:
len(labels),len(face_img)

(3261, 3261)

In [23]:
# 创建数据输入管道

face_img2 = [cv2.imread(path) for path in face_img]
face_img2 = [cv2.resize(img, (48, 48)) for img in face_img2]
face_img2 = [x/255 for x in face_img2]

In [26]:
# 创建dataset
img_ds = tf.data.Dataset.from_tensor_slices(face_img2)
labels_ds = tf.data.Dataset.from_tensor_slices(labels)

In [27]:
data = tf.data.Dataset.zip((img_ds, labels_ds))

In [28]:
data

<ZipDataset shapes: ((48, 48, 3), ()), types: (tf.float64, tf.int32)>

In [29]:
# 划分训练集和测试集
image_count = len(face_img)
test_count = int(image_count*0.2)
train_count = image_count-test_count

train_data = data.skip(test_count)
test_data = data.take(test_count)

train_count

2609

In [30]:
train_data = data.repeat().shuffle(train_count).batch(8)
test_data = test_data.batch(8)

In [3]:
# 构造模型
model = tf.keras.Sequential() #顺序模型

In [4]:
model.add(tf.keras.layers.Conv2D(64, (3,3), input_shape=(48, 48, 3), activation='relu'))

In [5]:
model.add(tf.keras.layers.Conv2D(64, (3,3), activation='relu'))

In [6]:
model.add(tf.keras.layers.MaxPooling2D())

In [7]:
model.add(tf.keras.layers.Conv2D(128, (3,3), activation='relu'))

In [8]:
model.add(tf.keras.layers.Conv2D(128, (3,3), activation='relu'))

In [40]:
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [41]:
# 配置模型
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001),
              loss='binary_crossentropy',
              metrics=['acc']
             )

In [42]:
callbacks = tf.keras.callbacks.ModelCheckpoint('M4.h5',save_best_only = True)

In [43]:
steps_per_epoch = train_count//8
validation_steps = test_count//8

In [44]:
history = model.fit(train_data, 
                    epochs=10, 
                    steps_per_epoch=steps_per_epoch,
                    validation_data=test_data,
                    validation_steps=validation_steps,  
                    callbacks=[callbacks]
)

Train for 326 steps, validate for 81 steps
Epoch 1/10
326/326 [==============================] - 20s 62ms/step - loss: 0.6167 - acc: 0.8980 - val_loss: 0.0736 - val_acc: 0.9645
Epoch 2/10
326/326 [==============================] - 5s 17ms/step - loss: 0.2052 - acc: 0.9379 - val_loss: 0.3146 - val_acc: 0.9738
Epoch 3/10
326/326 [==============================] - 6s 18ms/step - loss: 0.1782 - acc: 0.9456 - val_loss: 0.0443 - val_acc: 0.9877
Epoch 4/10
326/326 [==============================] - 6s 17ms/step - loss: 0.1445 - acc: 0.9525 - val_loss: 0.1661 - val_acc: 0.9306
Epoch 5/10
326/326 [==============================] - 6s 17ms/step - loss: 0.1863 - acc: 0.9425 - val_loss: 0.0932 - val_acc: 0.9722
Epoch 6/10
326/326 [==============================] - 6s 18ms/step - loss: 0.1665 - acc: 0.9471 - val_loss: 0.0735 - val_acc: 0.9769
Epoch 7/10
326/326 [==============================] - 6s 17ms/step - loss: 0.1338 - acc: 0.9509 - val_loss: 0.0802 - val_acc: 0.9753
Epoch 8/10
326/326 [=====